In [1]:
import re, json, pathlib, logging, time, argparse, pprint 
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import postprocess.postprocess as pspr
import keyword


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [2]:
with rsrc.path("keywords", "keywords_expanded_manually_reordered.xlsx") as res_path:
    keywords_path = res_path

keywords = pd.read_excel(res_path, index_col=0)

stop_words = set(stopwords.words('english'))-set(['no','not','nor'])
stop_words.remove('all')

keywd_cols = sorted(set(keywords.columns.to_list()) - set(['Goal', 'Target']))

def stem_text(my_keyword):
    my_keyword = plmp.preprocess_text(str(my_keyword), stop_words)
    return my_keyword

applymap_time = time.time()
keywords[keywd_cols]= keywords[keywd_cols].applymap(stem_text, na_action='ignore')
labels = keywords[['Goal', 'Target']]
print(time.time()-applymap_time)

# apply_time = time.time()
# keywords[keywd_cols]= keywords[keywd_cols].apply(plmp.preprocess_text, args=(stop_words,), axis=0)


keywords.head()



0.40753769874572754


,Goal,Target,0,1,2,3,4,5,6,7,...,47,48,49,50,51,52,53,54,55,56
Index,,,,,,,,,,,,,,,,,,,,,
1,SDG 1,1.0,sustain develop goal 1,sdg 1,end poverti,reduc poverti,fight poverti,end global poverti,poverti reduct,poverti depriv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SDG 1,1.1,intern poverti line,extrem poverti,abject poverti,absolut poverti,destitut,penuri,sever depriv,help peopl suffer food materi depriv,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SDG 1,1.2,sever materi depriv peopl,low work intens,nation poverti line,social exclus,incom poverti,combat child poverti,children poverti,address child poverti,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SDG 1,1.3,social risk,social protect,right person disabl,right unemploy person,right older person,right pregnant women,right work-injuri victim,right poor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SDG 1,1.4,tenur right,equal right econom resourc,control land,access inherit,right land,access properti,access natur resourc,access new technolog,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
labels.head()

,Goal,Target
Index,,
1,SDG 1,1.0
2,SDG 1,1.1
3,SDG 1,1.2
4,SDG 1,1.3
5,SDG 1,1.4


In [4]:
#Load text

input = pathlib.Path('input/52019DC0640_stemmed')

files = sorted(input.glob('**/*.*'))

print(files)

text_ls = {}

for _file in files:
    with open(_file, 'r') as _f:
        text_ls[_file.stem]=_f.read()

[PosixPath('input/52019DC0640_stemmed/CELEX_52019DC0640_EN_TXT_html_stemmed.txt'), PosixPath('input/52019DC0640_stemmed/cellar_b828d165-1c22-11ea-8c1f-01aa75ed71a1_0002_01_DOC_1_doc_stemmed.txt'), PosixPath('input/52019DC0640_stemmed/cellar_b828d165-1c22-11ea-8c1f-01aa75ed71a1_0002_01_DOC_2_doc_stemmed.txt'), PosixPath('input/52019DC0640_stemmed/cellar_b828d165-1c22-11ea-8c1f-01aa75ed71a1_0002_02_DOC_1_pdf_stemmed.txt'), PosixPath('input/52019DC0640_stemmed/cellar_b828d165-1c22-11ea-8c1f-01aa75ed71a1_0002_02_DOC_2_pdf_stemmed.txt')]


In [5]:
text_df = []

texts = list(text_ls.values())

keyword_matrix = keywords[keywd_cols]#.to_numpy()

count_matrix = keyword_matrix.applymap(lambda keyword: texts[0].count(keyword), na_action='ignore')#.to_numpy()

detected_keywords = keyword_matrix[count_matrix > 0]
count_matrix.replace(0, np.nan, inplace=True)


# labels = keywords['Target_keys'].iloc[:,0]

# counted_keywords = count_matrix.mask(count_matrix > 0, keyword_matrix)
# counted_keywords = counted_keywords.apply(lambda keyword : ', '.join(keyword.astype(str)))
# count_matrix = count_matrix.replace(0, '')
# counted_keywords.to_excel('key_col.xlsx')
# count_matrix.to_excel('mask.xlsx')

In [6]:
keyword_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
Index,,,,,,,,,,,,,,,,,,,,,
1,sustain develop goal 1,sdg 1,end poverti,reduc poverti,fight poverti,end global poverti,poverti reduct,poverti depriv,combact poverti,erad poverti,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,intern poverti line,extrem poverti,abject poverti,absolut poverti,destitut,penuri,sever depriv,help peopl suffer food materi depriv,extrem poor,incid poverti,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sever materi depriv peopl,low work intens,nation poverti line,social exclus,incom poverti,combat child poverti,children poverti,address child poverti,risk poverti,fight child poverti,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,social risk,social protect,right person disabl,right unemploy person,right older person,right pregnant women,right work-injuri victim,right poor,right vulner,hire subsidi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tenur right,equal right econom resourc,control land,access inherit,right land,access properti,access natur resourc,access new technolog,access financi servic,access microfin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
count_matrix=pd.merge(labels, count_matrix, left_index=True, right_index=True)
count_matrix.head()

,Goal,Target,0,1,2,3,4,5,6,7,...,47,48,49,50,51,52,53,54,55,56
Index,,,,,,,,,,,,,,,,,,,,,
1,SDG 1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SDG 1,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SDG 1,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SDG 1,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SDG 1,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def join_str(numpy_array):
    numpy_array=numpy_array[~np.isnan(numpy_array)]
    list_of_keys= np.char.join(', ', numpy_array)
    return type(numpy_array)

In [14]:
feh=detected_keywords[keywd_cols].apply(join_str, raw=True)

ValueError: invalid __array_struct__

In [ ]:
df = pd.read_excel('output/The_EU_Green_Deal_/output/6-results/results_.xlsx', sheet_name='priorities', index_col=0)

pp_colors={'Human Development':'#FECA4D',
'Growth Jobs':'#FABE13',
'Green Deal':'#73936F',
'Partnership':'#1A6835',
'Digital':'#72ABDC',
'Governance':'#E2809B',
'Digitalisation':'#4D9CD5',
'Migration':'#DE6189'
}

df.rename(columns = {'priority':'name', 'Count':'size'}, inplace = True)
df['goal_color']=df['name'].map(pp_colors)
df.to_dict('index')


In [ ]:
#Expand keywords

input = 'keywords/keywords_copia.xlsx'
keywords = pd.ExcelFile(input)
filename = pathlib.Path(input).stem
timestamp =  dt.datetime.now().isoformat(timespec='seconds').replace(':','').replace('T','_T')
filename += f'_{timestamp}'

output = f'{filename}.xlsx'

#sheets = a list of sheets where keywords are
#keys_df = [keywords.parse(sheet) for sheet in sheets]
keys_dfs = [keywords.parse('Target_keys'), keywords.parse('Goal_keys'), keywords.parse('MOI')]

for index, df in enumerate(keys_dfs):
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    df['Keys']=df['Keys'].apply(lambda keywords: re.sub(';;', ';', keywords))
    ex_df=df['Keys'].str.split(';', expand=True)
    lab_df=df.drop(columns='Keys')
    keys_dfs[index]=lab_df.join(ex_df)
    #df['Keys']=df['Keys'].apply(lambda keywords: re.sub(r'[^a-zA-Z0-9; -.]+', ';', keywords))

ex_df.head()

keys_dfs.append(keywords.parse('developing_countries'))

target_df, goal_df, devco_df, devco_names = keys_dfs

In [ ]:
target_df.head()

In [ ]:
ex_target_df=target_df['Keys'].str.split(';', expand=True)
lab_target_df=target_df.drop(columns='Keys')
lab_target_df.head()

In [ ]:
with pd.ExcelWriter(output, engine='xlsxwriter') as writer: # 'keywords_copy_for_transaltion.xlsx'
    target_df.to_excel(writer, sheet_name='Target_keys')
    goal_df.to_excel(writer, sheet_name='Goal_keys')
    devco_df.to_excel(writer, sheet_name='MOI')
    devco_names.to_excel(writer, sheet_name='developing_countries')

In [ ]:
#Load expand keywords

input = 'keywords/keywords_expanded.xlsx'

keywords = pd.ExcelFile(input)

sheets = keywords.sheet_names
print(f"Reading keywords dataset in {input}.\nSheets: {', '.join(sheets)}")

keywords = {sheet : keywords.parse(sheet, index_col=0) for sheet in sheets}
keywords_sheets = ['Target_keys', 'Goal_keys', 'MOI']

#remove all from stop_words to keep in keywords
stop_words = set(stopwords.words('english'))-set(['no','not','nor'])
stop_words.remove('all')

for sheet in keywords_sheets:
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: re.sub(';$', '', keywords))
    # keywords[sheet]['Keys'] = keywords[sheet]['Keys'].apply(lambda keywords: [plmp.preprocess_text(keyword, stop_words) for keyword in keywords.split(';')])
    keywords[sheet].iloc[:,2:]=keywords[sheet].iloc[:,2:].applymap(lambda keyword: plmp.preprocess_text(str(keyword), stop_words))

In [ ]:
keywords['Target_keys'].head()

In [ ]:
#Load text

input = pathlib.Path('input/52019DC0640_stemmed')

files = sorted(input.glob('**/*.*'))

print(files)

text_ls = {}

for _file in files:
    with open(_file, 'r') as _f:
        text_ls[_file.stem]=_f.read()

In [ ]:
def count_keyword(text, keyword):

count = text.count(keyword)


In [ ]:
keywords['Target_keys']

In [ ]:
text_df = []

texts = list(text_ls.values())

keyword_matrix = keywords['Target_keys'].iloc[:,2:]#.to_numpy()
count_matrix = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: texts[0].count(keyword))#.to_numpy()
labels = keywords['Target_keys'].iloc[:,0]

counted_keywords = count_matrix.mask(count_matrix > 0, keyword_matrix)
counted_keywords = counted_keywords.apply(lambda keyword : ', '.join(keyword.astype(str)))
count_matrix = count_matrix.replace(0, '')
counted_keywords.to_excel('key_col.xlsx')
count_matrix.to_excel('mask.xlsx')


# for name, text in text_ls.items():
#     count_df = keywords['Target_keys'].iloc[:,2:].applymap(lambda keyword: text.count(keyword))
#     #count_matrix = count_df#.to_numpy()
#     count_df = count_df.join(labels, how='inner')
#     count_df['Sum_counts'] = count_df.sum(axis=0)
#     keywords_ = keyword_matrix[count_matrix>0]
#     #count_df = count_df[count_df['Sum_counts'] > 0]
#     # #df.replace(0, None, inplace=True)
#     #df.insert(loc=1, column='Label', value=keywords['Target_keys'].iloc[:,1])
#     # df.insert(loc=2, coulmn='Count_keys', value=df.apply(lambda x: sum(x),axis=1))
#     # df.insert(loc=3, column='Diff_keys', value=df.apply(lambda x:count(x),axis=1))
#     count_df.to_excel(f'test_{name}.xlsx')
#     np.savetxt(f'test_{name}.csv', count_matrix, delimiter=',', fmt='%i')
#     np.savetxt(f'test_{name}_keys.csv', keywords_, delimiter=',',fmt='%s')
#     # np.savetxt('test.csv', count_matrix, delimiter=',')
#     text_df.append(count_df)

In [ ]:
keyword_matrix.head()

In [ ]:
count_matrix=count_matrix.mask(count_matrix > 1, -1)
count_matrix.to_excel('mask.xlsx')

In [ ]:
input = 'output/results_green_deal_test.xlsx'

result_df = pd.ExcelFile(input)

print(result_df.sheet_names)

targets, goals, total = result_df.parse('target_count'), result_df.parse('goal_overview'), result_df.parse('total_count_(goals_+_targets)')

# print(result_df.sheet_names)

In [ ]:
targets.rename(columns={'Count':'Target_Count', 'Num_keys':'Tgt_nr.keys'}, inplace=True) 
targets.groupby(['Policy','Goal','Target']).sum()
dat_target_join_keys = raw_df.groupby(['Policy', 'Target'])["Keyword"].apply(lambda x : " - ".join(x.astype(str))).reset_index()

In [ ]:
targets.head

In [ ]:
goals.rename(columns = {'Count':'Goal_Count', 'Goal_nr.keys'}, inplace = True) 
goals.groupby(['Policy','Goal']).sum()